# Activity Log Routes

> routes for interacting with the activity log


In [16]:
# | default_exp routes.activity_log

In [17]:
# | exporti

import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


In [18]:
# | export
async def get_activity_log_object_types(auth: dmda.DomoAuth) -> rgd.ResponseGetData:
    """retrieves a list of valid objectTypes that can be used to search the activity_log API"""

    url = "https://domo-community.domo.com/api/audit/v1/user-audits/objectTypes"

    return await gd.get_data(url=url, method="GET", auth=auth)

#### sample implementation of get_activity_log_object_types


In [19]:
import os
import pandas as pd

try:
    token_auth = dmda.DomoTokenAuth(
        domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
        domo_instance="domo-community",
    )

    al_objecttype_res = await get_activity_log_object_types(auth=token_auth)

    print(pd.DataFrame(al_objecttype_res.response))

    # use to update `ActivityLog_ObjectType` enum
    # [ f"{objecttype.get('translation').upper().replace(' ', '_')} = '{objecttype.get('type')}'" for objecttype in  al_objecttype_res.response]

except Exception as e:
    print(e)


adjusting num_stacks_to_drop, consider revising `get_traceback` call
{16, 3, 12}
                      type             translation
0             ACCESS_TOKEN            Access token
1                  ACCOUNT                 Account
2             ACTIVITY_LOG            Activity log
3                    ALERT                   Alert
4                      APP                     App
..                     ...                     ...
76           Workbench_JOB           Workbench job
77      Workbench_SCHEDULE      Workbench schedule
78       WORKFLOW_INSTANCE       Workflow Instance
79          WORKFLOW_MODEL          Workflow Model
80  WORKFLOW_MODEL_VERSION  Workflow Model Version

[81 rows x 2 columns]


## [Error] defining search_activity_log fail

In [20]:
# | export
import domolibrary.client.DomoError as de

class search_activity_log_error(de.DomoError):
    def __init__(
        self, 
        domo_instance, 
        status, 
        message, 
        entity_id, 
    ):
        super().__init__(
            function_name=function_name,
            entity_id=entity_id,
            domo_instance=domo_instance,
            status=status,
            message=message,
        )



## [update to include Error] testing test 

In [21]:
# | export
async def search_activity_log(
) -> rgd.ResponseGetData:
    """loops over activity log api to retrieve audit logs"""
    
    try: #adding try to test first. 
        is_close_session = False

        if not session: #jae's code 
            session = httpx.AsyncClient()
            is_close_session = True

        url = f"https://{auth.domo_instance}.domo.com/api/audit/v1/user-audits"

        if object_type and object_type != 'ACTIVITY_LOG':
            url = f"{url}/objectTypes/{object_type}"

        fixed_params = {"end": end_time, "start": start_time}

        offset_params = {
            "offset": "offset",
            "limit": "limit",
        }

        def arr_fn(res) -> list[dict]: #jae's code
            return res.response

        res = await gd.looper(  #jaes code 
            auth=auth,
            method="GET",
            url=url,
            arr_fn=arr_fn,
            fixed_params=fixed_params,
            offset_params=offset_params,
            session=session,
            maximum=maximum,
            debug_loop=debug_loop,
            debug_api=debug_api,
        )

    except Exception as e: #my attempt at adding a tester 
        raise search_activity_log_error( #pointing this to the function above "blahblah_error(de.DomoError)"
            domo_instance=dmda.DomoAuth,
            status="error status",
            message=str(e),
            entity_id="entity_id"
        )

    finally:
        if is_close_session: #jae's code 
            await session.aclose()

    return res


In [23]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

page_res = await search_activity_log(
    object_type= 'PAGE',
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=16,
    debug_loop=False,
    debug_api = False
)

pd.DataFrame(page_res.response[0:5])


,objectId,objectName
0,-100000,Overview
1,-100001,Alerts
2,-100002,Shared
3,-100003,Favorites
4,-100012,CEO


#### sample implementation of search_activity_log


In [24]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

activity_log_res = await search_activity_log(
    object_type= 'ACTIVITY_LOG',
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=10,
    debug_loop=False,
    debug_api=False,
)

pd.DataFrame(activity_log_res.response[0:5])

,userName,userId,userType,actorName,actorId,actionType,objectName,objectId,objectType,additionalComment,time,eventText,clientId,authenticationMethod
0,Bryan Van Kampen,587894148,USER,,0,VIEWED,Workbench Monitoring,1416746303,PAGE,Bryan Van Kampen viewed page Workbench Monitor...,1696266186196,Viewed page,None,DEV_TOKEN
1,Bryan Van Kampen,587894148,USER,,0,VIEWED,Welcome,30507758,PAGE,Bryan Van Kampen viewed page Welcome.,1696266185780,Viewed page,None,DEV_TOKEN
2,Bryan Van Kampen,587894148,USER,,0,VIEWED,Virtual User Group,138471350,PAGE,Bryan Van Kampen viewed page Virtual User Group.,1696266184259,Viewed page,None,DEV_TOKEN
3,Bryan Van Kampen,587894148,USER,,0,VIEWED,Time Zone Handling,950006533,PAGE,Bryan Van Kampen viewed page Time Zone Handling.,1696266182840,Viewed page,None,DEV_TOKEN
4,Bryan Van Kampen,587894148,USER,,0,VIEWED,Text Analytics,225044251,PAGE,Bryan Van Kampen viewed page Text Analytics.,1696266182514,Viewed page,None,DEV_TOKEN


In [25]:
# | hide
import nbdev

nbdev.nbdev_export()
